In [70]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import torch
import torch.nn as nn
import lightning as pl
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from lightning.pytorch.loggers import WandbLogger
import wandb

In [7]:
file_path = r"C:\Users\KIIT\Desktop\MiniProject\train.csv"
train = pd.read_csv(file_path)

In [9]:
train

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False
...,...,...,...,...,...
421565,45,98,2012-09-28,508.37,False
421566,45,98,2012-10-05,628.10,False
421567,45,98,2012-10-12,1061.02,False
421568,45,98,2012-10-19,760.01,False


In [11]:
train = train[train["Store"] == 1]

In [15]:
file_path = r"C:\Users\KIIT\Desktop\MiniProject\features.csv"
features = pd.read_csv(file_path)

In [17]:
features

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False
...,...,...,...,...,...,...,...,...,...,...,...,...
8185,45,2013-06-28,76.05,3.639,4842.29,975.03,3.00,2449.97,3169.69,NaN,NaN,False
8186,45,2013-07-05,77.50,3.614,9090.48,2268.58,582.74,5797.47,1514.93,NaN,NaN,False
8187,45,2013-07-12,79.37,3.614,3789.94,1827.31,85.72,744.84,2150.36,NaN,NaN,False
8188,45,2013-07-19,82.84,3.737,2961.49,1047.07,204.19,363.00,1059.46,NaN,NaN,False


In [19]:
features = features[features["Store"] == 1]

In [21]:
train = train.merge(features, on="Date")

In [23]:
train

,Store_x,Dept,Date,Weekly_Sales,IsHoliday_x,Store_y,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y
0,1,1,2010-02-05,24924.50,False,1,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,1,2010-02-12,46039.49,True,1,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,1,2010-02-19,41595.55,False,1,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,1,2010-02-26,19403.54,False,1,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,1,2010-03-05,21827.90,False,1,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10239,1,99,2012-08-31,20.06,False,1,80.49,3.638,21442.73,102.32,21.70,13209.64,3032.96,222.305480,6.908,False
10240,1,99,2012-09-07,0.05,True,1,83.96,3.730,5204.68,35.74,50.94,4120.32,2737.17,222.439015,6.908,True
10241,1,99,2012-09-14,0.03,False,1,74.97,3.717,17212.52,7.00,18.79,1523.11,7992.72,222.582019,6.908,False
10242,1,99,2012-10-05,635.00,False,1,68.55,3.617,8077.89,NaN,18.22,3617.43,3626.14,223.181477,6.573,False


In [25]:
train.isna().sum()

Store_x            0
Dept               0
Date               0
Weekly_Sales       0
IsHoliday_x        0
Store_y            0
Temperature        0
Fuel_Price         0
MarkDown1       6587
MarkDown2       7229
MarkDown3       6656
MarkDown4       6587
MarkDown5       6587
CPI                0
Unemployment       0
IsHoliday_y        0
dtype: int64

In [27]:
train = train.fillna(0)

In [29]:
train

,Store_x,Dept,Date,Weekly_Sales,IsHoliday_x,Store_y,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday_y
0,1,1,2010-02-05,24924.50,False,1,42.31,2.572,0.00,0.00,0.00,0.00,0.00,211.096358,8.106,False
1,1,1,2010-02-12,46039.49,True,1,38.51,2.548,0.00,0.00,0.00,0.00,0.00,211.242170,8.106,True
2,1,1,2010-02-19,41595.55,False,1,39.93,2.514,0.00,0.00,0.00,0.00,0.00,211.289143,8.106,False
3,1,1,2010-02-26,19403.54,False,1,46.63,2.561,0.00,0.00,0.00,0.00,0.00,211.319643,8.106,False
4,1,1,2010-03-05,21827.90,False,1,46.50,2.625,0.00,0.00,0.00,0.00,0.00,211.350143,8.106,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10239,1,99,2012-08-31,20.06,False,1,80.49,3.638,21442.73,102.32,21.70,13209.64,3032.96,222.305480,6.908,False
10240,1,99,2012-09-07,0.05,True,1,83.96,3.730,5204.68,35.74,50.94,4120.32,2737.17,222.439015,6.908,True
10241,1,99,2012-09-14,0.03,False,1,74.97,3.717,17212.52,7.00,18.79,1523.11,7992.72,222.582019,6.908,False
10242,1,99,2012-10-05,635.00,False,1,68.55,3.617,8077.89,0.00,18.22,3617.43,3626.14,223.181477,6.573,False


In [31]:
train.columns.to_list()

['Store_x',
 'Dept',
 'Date',
 'Weekly_Sales',
 'IsHoliday_x',
 'Store_y',
 'Temperature',
 'Fuel_Price',
 'MarkDown1',
 'MarkDown2',
 'MarkDown3',
 'MarkDown4',
 'MarkDown5',
 'CPI',
 'Unemployment',
 'IsHoliday_y']

In [37]:
data_parts = train.groupby(["Store_x", "Dept"])
new_df_parts = []
for idx, data in data_parts:
    for lag in range(1, 4):
        data[f'lag_{lag}'] = data['Weekly_Sales'].shift(lag)  # Replace 'target_variable'

    # Rolling Window Features
    for window in [7, 14, 28]:
        data[f'rolling_mean_{window}'] = data['Weekly_Sales'].rolling(window=window).mean()
        data[f'rolling_std_{window}'] = data['Weekly_Sales'].rolling(window=window).std()
        data[f'rolling_min_{window}'] = data['Weekly_Sales'].rolling(window=window).min()
        data[f'rolling_max_{window}'] = data['Weekly_Sales'].rolling(window=window).max()
    for span in [5, 10, 20]:
        data[f'ewm_{span}'] = data['Weekly_Sales'].ewm(span=span).mean()

    # Expanding Window Features
    data['expanding_mean'] = data['Weekly_Sales'].expanding().mean()
    data['expanding_std'] = data['Weekly_Sales'].expanding().std()

    # Handle Missing Values
    data.fillna(method='ffill', inplace=True)
    new_df_parts.append(data)

C:\Users\KIIT\AppData\Local\Temp\ipykernel_10428\2739975061.py:21: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)
C:\Users\KIIT\AppData\Local\Temp\ipykernel_10428\2739975061.py:21: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)
C:\Users\KIIT\AppData\Local\Temp\ipykernel_10428\2739975061.py:21: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)
C:\Users\KIIT\AppData\Local\Temp\ipykernel_10428\2739975061.py:21: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  data.fillna(method='ffill', inplace=True)
C:\Users\KII

In [39]:
new_df_parts[0]

,Store_x,Dept,Date,Weekly_Sales,IsHoliday_x,Store_y,Temperature,Fuel_Price,MarkDown1,MarkDown2,...,rolling_max_14,rolling_mean_28,rolling_std_28,rolling_min_28,rolling_max_28,ewm_5,ewm_10,ewm_20,expanding_mean,expanding_std
0,1,1,2010-02-05,24924.50,False,1,42.31,2.572,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,24924.500000,24924.500000,24924.500000,24924.500000,NaN
1,1,1,2010-02-12,46039.49,True,1,38.51,2.548,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,37593.494000,36537.744500,36009.869750,35481.995000,14930.552614
2,1,1,2010-02-19,41595.55,False,1,39.93,2.514,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,39489.204737,38570.948704,38060.898052,37519.846667,11131.900957
3,1,1,2010-02-26,19403.54,False,1,46.63,2.561,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,31145.928615,32256.141530,32674.807250,32990.770000,12832.106391
4,1,1,2010-03-05,21827.90,False,1,46.50,2.625,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,27568.865498,29262.477009,30051.032691,30758.196000,12182.739805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,1,1,2012-09-28,18947.81,False,1,76.08,3.666,3666.27,7.64,...,19616.22,19884.016429,8445.936215,15722.82,57592.12,18588.028445,18095.479149,18614.901097,22468.781439,9986.247452
139,1,1,2012-10-05,21904.47,False,1,68.55,3.617,8077.89,0.00,...,21904.47,19876.758214,8444.041576,15722.82,57592.12,19693.508963,18788.022940,18928.193631,22464.750643,9950.375157
140,1,1,2012-10-12,22764.01,False,1,62.99,3.601,2086.18,0.00,...,22764.01,19655.727857,8277.014816,15722.82,57592.12,20717.009309,19510.929679,19293.509747,22466.873050,9914.806446
141,1,1,2012-10-19,24185.27,False,1,67.97,3.594,950.33,0.00,...,24185.27,18462.626071,3806.580249,15722.82,34684.21,21873.096206,20360.809737,19759.391990,22478.974437,9880.637364


In [41]:
new_df_parts[0].columns

Index(['Store_x', 'Dept', 'Date', 'Weekly_Sales', 'IsHoliday_x', 'Store_y',
       'Temperature', 'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3',
       'MarkDown4', 'MarkDown5', 'CPI', 'Unemployment', 'IsHoliday_y', 'lag_1',
       'lag_2', 'lag_3', 'rolling_mean_7', 'rolling_std_7', 'rolling_min_7',
       'rolling_max_7', 'rolling_mean_14', 'rolling_std_14', 'rolling_min_14',
       'rolling_max_14', 'rolling_mean_28', 'rolling_std_28', 'rolling_min_28',
       'rolling_max_28', 'ewm_5', 'ewm_10', 'ewm_20', 'expanding_mean',
       'expanding_std'],
      dtype='object')

In [43]:
train = pd.concat(new_df_parts)

In [45]:
train

,Store_x,Dept,Date,Weekly_Sales,IsHoliday_x,Store_y,Temperature,Fuel_Price,MarkDown1,MarkDown2,...,rolling_max_14,rolling_mean_28,rolling_std_28,rolling_min_28,rolling_max_28,ewm_5,ewm_10,ewm_20,expanding_mean,expanding_std
0,1,1,2010-02-05,24924.50,False,1,42.31,2.572,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,24924.500000,24924.500000,24924.500000,24924.500000,NaN
1,1,1,2010-02-12,46039.49,True,1,38.51,2.548,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,37593.494000,36537.744500,36009.869750,35481.995000,14930.552614
2,1,1,2010-02-19,41595.55,False,1,39.93,2.514,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,39489.204737,38570.948704,38060.898052,37519.846667,11131.900957
3,1,1,2010-02-26,19403.54,False,1,46.63,2.561,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,31145.928615,32256.141530,32674.807250,32990.770000,12832.106391
4,1,1,2010-03-05,21827.90,False,1,46.50,2.625,0.00,0.00,...,NaN,NaN,NaN,NaN,NaN,27568.865498,29262.477009,30051.032691,30758.196000,12182.739805
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10239,1,99,2012-08-31,20.06,False,1,80.49,3.638,21442.73,102.32,...,240.0,353.581429,602.039659,0.01,2400.0,13.389095,57.065755,159.690017,321.523939,561.181515
10240,1,99,2012-09-07,0.05,True,1,83.96,3.730,5204.68,35.74,...,240.0,351.797500,603.045694,0.01,2400.0,8.942725,46.687956,143.962826,312.068824,555.356727
10241,1,99,2012-09-14,0.03,False,1,74.97,3.717,17212.52,7.00,...,240.0,333.584286,605.785535,0.01,2400.0,5.971815,38.197128,129.829390,303.153429,549.665202
10242,1,99,2012-10-05,635.00,False,1,68.55,3.617,8077.89,0.00,...,635.0,351.977143,606.877740,0.01,2400.0,215.647973,146.785878,179.288221,312.371389,544.571800


In [47]:
val = train[train["Date"] >= "2012-01-01"]
train = train[train["Date"] < "2012-01-01"]

In [49]:
train = train.drop(columns=["Date"]).astype("float")
val = val.drop(columns=["Date"]).astype("float")

In [51]:
tr_columns = train.columns

In [53]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
train = scaler.fit_transform(train)
val = scaler.transform(val)

In [55]:
train = pd.DataFrame(train, columns=tr_columns)
val = pd.DataFrame(val, columns=tr_columns)

In [57]:
train_x = train.drop(columns=["Weekly_Sales", "Store_y"]).iloc[:-1]
train_y = train["Weekly_Sales"].iloc[1:]
val_x = val.drop(columns=["Weekly_Sales", "Store_y"]).iloc[:-1]
val_y = val["Weekly_Sales"].iloc[1:]

In [59]:
train_x = train_x.fillna(0)
val_x = val_x.fillna(0)

In [62]:
model = RandomForestRegressor(n_estimators=100, random_state=42) 
model.fit(train_x, train_y)

RandomForestRegressor(random_state=42)

In [63]:
pred_y = model.predict(train_x)
mse = mean_squared_error(train_y, pred_y)

print('Train Mean Squared Error:', mse)

Train Mean Squared Error: 0.0001432419577933629


In [66]:
pred_y = model.predict(val_x)
mse = mean_squared_error(val_y, pred_y)

print('Val Mean Squared Error:', mse)

Val Mean Squared Error: 0.0022758114768575326


In [68]:
!pip install --upgrade lightning wandb

   ---------------------------------------- 0.0/815.2 kB ? eta -:--:--
   ---------------------------------------- 815.2/815.2 kB 5.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/20.2 MB ? eta -:--:--
   ------------ --------------------------- 6.6/20.2 MB 31.0 MB/s eta 0:00:01
   ----------------------- ---------------- 11.8/20.2 MB 27.4 MB/s eta 0:00:01
   ------------------------ --------------- 12.6/20.2 MB 26.3 MB/s eta 0:00:01
   ------------------------ --------------- 12.6/20.2 MB 26.3 MB/s eta 0:00:01
   ------------------------ --------------- 12.6/20.2 MB 26.3 MB/s eta 0:00:01
   ------------------------ --------------- 12.6/20.2 MB 26.3 MB/s eta 0:00:01
   ----------------------------- ---------- 14.7/20.2 MB 9.7 MB/s eta 0:00:01
   ------------------------------ --------- 15.2/20.2 MB 9.5 MB/s eta 0:00:01
   ------------------------------ --------- 15.2/20.2 MB 9.5 MB/s eta 0:00:01
   ------------------------------ --------- 15.2/20.2 MB 9.5 MB/s eta 0:

In [72]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
train = scaler.fit_transform(train)
val = scaler.transform(val)

In [74]:
train

array([[0.        , 0.        , 0.12607961, ..., 0.16626042, 0.17904964,
               nan],
       [0.        , 0.        , 0.2293145 , ..., 0.23868926, 0.25340857,
        0.53050726],
       [0.        , 0.        , 0.2075873 , ..., 0.25209013, 0.26776165,
        0.39553487],
       ...,
       [0.        , 1.        , 0.01062418, ..., 0.0071291 , 0.00653179,
        0.02457131],
       [0.        , 1.        , 0.01331322, ..., 0.00819485, 0.00725102,
        0.02723259],
       [0.        , 1.        , 0.00617503, ..., 0.0079288 , 0.0071888 ,
        0.02627012]])

In [82]:

class TimeSeriesDataset(torch.utils.data.Dataset):
    def __init__(self, data, seq_len):
        self.seq_len = seq_len
        self.data = []
        self.targets = []

        # Convert data to DataFrame if not already
        data = pd.DataFrame(data)

        # Verify number of columns before renaming
        expected_columns = ['Store_x','Dept','Weekly_Sales','IsHoliday_x','Store_y',
                            'Temperature','Fuel_Price','MarkDown1','MarkDown2','MarkDown3',
                            'MarkDown4','MarkDown5','CPI','Unemployment','IsHoliday_y']

        if len(data.columns) >= len(expected_columns):
            data = data.iloc[:, :len(expected_columns)]  # Keep only necessary columns
            data.columns = expected_columns
        else:
            raise ValueError(f"Expected at least {len(expected_columns)} columns, but got {len(data.columns)}")

        # Group by 'Store_x' and 'Dept'
        data_parts = data.groupby(["Store_x", "Dept"])

        for _, group_data in data_parts:
            for i in range(len(group_data) - self.seq_len):
                self.data.append(torch.tensor(group_data.iloc[i:i+self.seq_len].drop(columns=["Weekly_Sales", "Store_y"]).values, dtype=torch.float32))
                self.targets.append(torch.tensor(group_data.iloc[i+self.seq_len]["Weekly_Sales"], dtype=torch.float32))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.targets[idx]  # Fixed return value


In [84]:
train_dataset = TimeSeriesDataset(train, 20)
val_dataset = TimeSeriesDataset(val, 20)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=False)

In [86]:
train_dataset[0]

(tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00, 1.2284e-01, 4.1637e-02, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 8.2506e-02, 1.0000e+00,
          0.0000e+00],
         [0.0000e+00, 0.0000e+00, 1.0000e+00, 5.5289e-02, 2.4408e-02, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 9.8357e-02, 1.0000e+00,
          1.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 8.0533e-02, 0.0000e+00, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0346e-01, 1.0000e+00,
          0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 1.9964e-01, 3.3740e-02, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0678e-01, 1.0000e+00,
          0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 1.9733e-01, 7.9684e-02, 0.0000e+00,
          0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.1010e-01, 1.0000e+00,
          0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 3.9804e-01, 1.0983e

In [90]:


class LSTMModel(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout, output_dim):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x, _ = self.lstm(x)  # LSTM output shape: (batch_size, seq_len, hidden_dim)
        x = self.fc(x[:, -1, :])  # Take last time step's output and pass through FC layer
        return x

    def training_step(self, batch, batch_idx):
        data, targets = batch
        outputs = self(data)
        targets = targets.view_as(outputs)  # Reshape targets to match outputs
        loss = nn.MSELoss()(outputs, targets)
        self.log('train_loss', loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        data, targets = batch
        outputs = self(data)
        targets = targets.view_as(outputs)  # Ensure matching shape
        loss = nn.MSELoss()(outputs, targets)
        self.log('val_loss', loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.0002)


In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
import pytorch_lightning as pl
from torch.utils.data import DataLoader, TensorDataset
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import WandbLogger

# Define LSTM Model
class LSTMModel(pl.LightningModule):
    def __init__(self, input_size, hidden_size, num_layers, output_size, learning_rate=1e-3):
        super(LSTMModel, self).__init__()
        self.save_hyperparameters()
        
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.criterion = nn.MSELoss()
        
    def forward(self, x):
        lstm_out, _ = self.lstm(x)  # LSTM output
        return self.fc(lstm_out[:, -1, :])  # Take only the last time step
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        self.log("val_loss", loss, prog_bar=True)
        return loss

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=self.hparams.learning_rate)

# Generate Random Data for Example
torch.manual_seed(42)
X_train = torch.rand(100, 10, 5)  # 100 samples, sequence length 10, features 5
y_train = torch.rand(100, 1)  # 100 output values
X_val = torch.rand(20, 10, 5)
y_val = torch.rand(20, 1)

# DataLoaders
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32)

# Callbacks & Logger
checkpoint_callback = ModelCheckpoint(monitor="val_loss", mode="min", save_top_k=1)
early_stopping_callback = EarlyStopping(monitor="val_loss", patience=5, mode="min")
wandb_logger = WandbLogger(project="lstm_project")

# Trainer
model = LSTMModel(input_size=5, hidden_size=32, num_layers=2, output_size=1, learning_rate=1e-3)
trainer = pl.Trainer(
    max_epochs=50,
    logger=wandb_logger,
    callbacks=[checkpoint_callback, early_stopping_callback]
)

# Train Model
trainer.fit(model, train_dataloader, val_dataloader)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



  | Name      | Type    | Params | Mode 
----------------------------------------------
0 | lstm      | LSTM    | 13.4 K | train
1 | fc        | Linear  | 33     | train
2 | criterion | MSELoss | 0      | train
----------------------------------------------
13.5 K    Trainable params
0         Non-trainable params
13.5 K    Total params
0.054     Total estimated model params size (MB)
3         Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

In [41]:
import os
print(os.listdir())  # Check files in your working directory


['.git', '.ipynb_checkpoints', 'check.ipynb', 'engineer_profile.json', 'features.csv', 'lightning_logs', 'lstmvsrt.ipynb', 'lstm_project', 'stores.csv', 'test.csv', 'train.csv', 'Untitled.ipynb', 'Untitled1.ipynb', 'walmart-recruiting-store-sales-forecasting', 'WalMart.ipynb', 'walmart_sales_predictions.csv', 'wandb']


In [29]:
wandb.finish()

epoch,▁▂▂▃▃▄▄▅▅▆▆▇▇▇█
train_loss,▁
trainer/global_step,▁▂▂▃▃▄▄▅▅▆▆▇▇▇█
val_loss,█▆▄▂▁▁▁▁▁▁▁▁▁▁
epoch,13
train_loss,0.08391
trainer/global_step,55
val_loss,0.05493
